# Parse

This notebook shows a simple example of extracting data from the npz file

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import cionic
from cionic import tools

## Load Collection

Download or construct the npz for the collection to be analyzed, and save it to the recordings directory.

As a sanity check print out the **positions**, **labels**, and **stream names**


In [ ]:
# param

npzpath = None
download = None
tokenpath = None
presentation = {}

In [ ]:
if download:
    cionic.auth(tokenpath=tokenpath)
    cionic.download_npz(npzpath, download)

npz = cionic.load_segmented(npzpath)
regs = tools.stream_regs(np.load(npzpath))  # can only be loaded from original npz
outpath = os.path.splitext(npzpath)[0] + '.pdf'

if 'position' in npz['segments'].dtype.names:
    print(f"Positions: {set(npz['segments']['position'])}")

if 'label' in npz['segments'].dtype.names:
    print(f"Labels: {set(npz['segments']['label'])}")

if 'stream' in npz['segments'].dtype.names:
    print(f"Stream: {set(npz['segments']['stream'])}")

# Using Parsed Data

The segmented npz file contains .npy files of time series data for each stream and label

The npz also contains a `segments` file that describes each of the segmets

in the example below we will search the segments file for EMG streams, display a subset of the available fields

and plot the first available stream

In [ ]:
# show available fields in npz segments file
df = pd.DataFrame(npz['segments'])
list(df.keys())

In [ ]:
# show EMG streams, displaying a subset of fields
emg_df = df[df['stream'] == 'emg'][
    ['label', 'position', 'start_s', 'end_s', 'segment_num', 'path']
]
emg_df

In [ ]:
# load .npy file for the first emg segment
#
# tools.load_streams takes any passed kwargs and filters the npz
#
# examples:
#     streams = tools.load_streams(npz, stream='emg', label=['unassisted','stimulated'])
#     streams = tools.load_streams(npz, stream='fquat', label=['bend','stand'])
#
# the result is a lit of stream dictionaries with stream, position, label, segment, and values
# where values is a numpy array of the time series data

path = emg_df.iloc[0].path
streams = tools.load_streams(npz, path=path)
streams

In [ ]:
# plot first stream channel 3

plt.plot(streams[0]['values']['c3'])
plt.show()